This notebooks provides an example of how to create plots displaying the results of the analysis.<br>
As a 4D analysis is made up of of several 2D interpolations (loops on time and depth), we can use the tools of the `pydiva2d` module to produce the figures for the contours, meshes and results.

In [1]:
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
import sys
sys.path.insert(0, "../")
import pydiva4d

In [4]:
pydiva4d.logger.setLevel(logging.DEBUG)

### Options for the plot

Define a directory for the figures

In [5]:
figdir = './figures/BlackSea/'
if not os.path.exists(figdir):
    os.makedirs(figdir)

Define the directory where Diva will be executed:

In [6]:
divamaindir = '/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/'
Diva4Ddirs = pydiva4d.Diva4DDirectories(divamaindir)

Generate the file names

In [7]:
Diva4Dfiles = pydiva4d.Diva4Dfiles(Diva4Ddirs.diva4d)

## Load the depth levels

In [19]:
contourdepth = pydiva4d.Contourdepth()
contourdepth.read_from(Diva4Dfiles.contourdepth)
contourdepth.depthlist

[30.0, 20.0, 10.0, 0.0]

## Loop on the contours

In [9]:
for idepth, depth in enumerate(contourdepth.depthlist):
    
    contourfile = os.path.join(Diva4Ddirs.diva4dparam, 'coast.cont.{0}'.format(str(10001 + idepth)))

    contour2D = pydiva4d.Diva2DContours()
    contour2D.read_from(contourfile)

    fig = plt.figure()
    contour2D.add_to_plot(color='k', linewidth=.5)
    plt.xlim(26., 42.)
    plt.ylim(40., 48.)
    plt.title("Contour at depth: {0} m".format(depth))
    plt.savefig(os.path.join(figdir, "BlackSea_contour{0}".format(idepth)))
    plt.close()

## Loop on the meshes

In [10]:
for idepth, depth in enumerate(contourdepth.depthlist):
    # Create the file names
    meshtopofile = os.path.join(Diva4Ddirs.diva4dmesh, "mesh.dat.{0}".format(str(10000 + idepth + 1)))
    meshfile = os.path.join(Diva4Ddirs.diva4dmesh, "meshtopo.{0}".format(str(10000 + idepth + 1)))
    
    # Mesh object
    Mesh = pydiva4d.Diva2DMesh()
    Mesh.read_from(meshfile, meshtopofile)
    
    # Make the plot
    fig = plt.figure()
    ax = plt.subplot(111)
    Mesh.add_to_plot(linewidth=0.1, color='k')
    plt.xlim(26., 42.)
    plt.ylim(40., 48.)
    plt.title("Mesh at depth: {0} m".format(depth))
    plt.savefig(os.path.join(figdir, "BlackSea_mesh{0}".format(idepth)))
    plt.close()

## Loop on the times and depth levels for the results

Load year and month periods:

In [15]:
Monthlist = pydiva4d.Monthlist()
Monthlist.read_from(Diva4Dfiles.monthlist)
Monthlist.monthlist

['0101', '0202', '0303', '']

In [16]:
Yearlist = pydiva4d.Yearlist()
Yearlist.read_from(Diva4Dfiles.yearlist)
Yearlist.yearlist

['19002009']

## Load the variables

In [17]:
Varlist = pydiva4d.Varlist()
Varlist.read_from(Diva4Dfiles.varlist)
Varlist.varlist

['Temperature', 'Salinity']

# Plot the fields

In [22]:
for variables in Varlist.varlist:
    for yearperiods in Yearlist.yearlist:
        for monthperiods in Monthlist.monthlist:
            for idepth, depthlevels in enumerate(contourdepth.depthlist):
                
                # Create file name
                resultfile = "{0}.{1}.{2}.{3}.anl.nc".format(variables, yearperiods,
                                                             monthperiods, str(10000 + idepth + 1))
                
                # Create figure name
                figname = ''.join((resultfile.replace('.', '_'), '.png'))
                figtitle = "{0}, Year: {1}, Months: {2}, Depth: {3} m".format(variables, yearperiods,
                                                                                       monthperiods, str(depthlevels))
                
                # Load the results
                if os.path.exists(os.path.join(Diva4Ddirs.diva4dfields, resultfile)):
                    Results = pydiva4d.Diva2DResults(os.path.join(Diva4Ddirs.diva4dfields, resultfile))
                    fig = plt.figure()
                    ax = plt.subplot(111)
                    resultplot = Results.add_to_plot()
                    plt.colorbar(resultplot)
                    plt.title(figtitle)
                    plt.savefig(os.path.join(figdir, figname))
                    plt.close()